In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import chi2_contingency
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from imblearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, \
    AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb 
from sklearn.preprocessing import FunctionTransformer

from sklearn.metrics import accuracy_score, \
    precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance


### Read Data

In [2]:
df = pd.read_csv('../data/Math.csv', sep=';', encoding='ISO-8859-1')